# Problem set 2

In [1]:
using Pkg
Pkg.activate(".")
# Pkg.add("Unitful")
# Pkg.add("Statistics")
# include("tests.jl")

  Activating project at `c:\Users\USAID\Documents\2023-2024\phys 218\NonLinearDynamics\Julia Practice`


## Problem 1: Type hierarchies

Create a function person_info(p) that takes in any type of a "person" and prints their name and potentially some extra information. 
* For a normal person, only the name is printed.
* If the person is a student, it should print their name and grade.
* If it is a group leader, then print their name and their group name.
* If the input to person_info is something other than a "person", it should error.

Solve this exercise without using a single if statement; only multiple dispatch. These kind of problems seem to be "natural" to solve using if statements. However using multiple dispatch instead makes the code clearer, more easy to extend and (typically) more performant, because of how Julia compiles code.

**Hint:** given the rules of the exercise, you do not actually need abstract types to solve it. However, it is in general good practice to use them.




In [2]:
# Fill in the structs below:
struct Person
    name::String
end

struct Student
    name::String
    grade::Number
end

struct Normal
    name::String
end

struct GroupLeader
    name::String
    group::String
end

In [3]:
# Fill in personal_info functions
person_info(p::Student) = print(p.name, ", ", p.grade)
person_info(p::Normal) = print(p.name)
person_info(p::GroupLeader) = print(p.name, ", ", p.group)
person_info(p::Person) = print(p.name)

person_info (generic function with 4 methods)

In [4]:
p1 = Person("John")
p2 = Student("Mary", 90)
p3 = GroupLeader("Tom", "Group A");

In [5]:
# Test them here
person_info(p3)

Tom, Group A

## Problem 2: Indexing of a range type


Create your own Range object, which is an efficient iterable container defined by a (start, step, stop), which gives numeric values incremented by step starting from start, while ensuring that the result never exceeds stop. You only need to extend (add a new method) to the Base.getindex(r::Range, i::Int) function for this exercise.

Test your range by initializing a range like r = Range(0, 0.5, 1), and checking if r[1], r[2], r[3] give you indeed 0.0, 0.5, 1.0 as they should.

This object can efficiently represent this range while storing only these 3 numbers, instead of all elements that theoretically belong to the range.

**Hints:**

* As a first step, define your Range struct, preferably by making it a parametric type of 1 type-parameter (think about how many fields it should have).
* The Julia syntax A[i] is translated to getindex(A, i). That's why you need to extend the Base.getindex function. Use the help functionality on getindex to learn more. Make sure you also take care of the case where the index is a slice (e.g. A[1:3]).
* Implement indexing for your Range type via extending getindex. Index 1 giving start. Be sure it errors for incorrect indices (where the value given by the range would exceed stop).
* To make sure all the inputs are of the correct type, you can use the `promote` function. For example, `promote(1, 2.0)` returns `(1.0, 2.0)`. You can use this to make sure that the inputs to your Range are all of the same type.
* Did you write generic code? Try to see if your Range works with the package Unitful. This package provides types that combines numbers with physical units (like Newtons). Load this package with using Unitful. Does iterating a Range(1u"kg", 0.1u"kg", 10u"kg") work as expected? Make the necessary adjustments so that it does.

In [2]:
# Fill in the struct here 
struct Range{T<: Int64} 
    start::T
    step::T
    stop::T
end

In [7]:
# Getindex function here
function Base.getindex(r::Range, i)
    output = []
    if i == Int64
        if i >= 1 && i < r.stop
            for o ∈ range(i)
                add!(output, v.start + (i-1)*v.step)
            end
        end
    end
    return output
 end


In [8]:
# Some tests here
# test_Range(Range);
r = Range(0.0, 0.5, 1.0)
Getindex(r, 1)

1.0

## Problem 3: Iteration of a range type

Continuing from the previous exercise, now make your Range object iterable by extending Base.iterate(r::Range, state = 1) as well as Base.length(r::Range). It will then be possible to do collect(range) or for a in range ... or [cos(r) for r in range].

In [9]:
# Fill function for iterate here

In [10]:
test_Range(Range, true);

UndefVarError: UndefVarError: `test_Range` not defined

## Problem 4: Advanced whackadoodle iterator  

Leveraging the experience of previous exercises, define an iterator object (and of course define iterate for it!) that has the following properties (make an iterator struct, don't create a vector!):

* It iterates for n elements. The first element is always rand().
* At the odd steps of the iteration it returns the previous element divided by rand().
* At the even steps of the iteration it returns the previous element multiplied by rand().
* Use this iterator object to efficiently study how its mean value and standard deviation behaves versus n, by using the functions mean, std (accessible after using Statistics).

**Hint:** if you implement this iterator correctly, then you should be able to give it directly to mean.

In [11]:
# Fill in the whackadoodle struct here

In [12]:
# Fill in the iterate function here

In [13]:
whacky = WhackyRange(10)
for i ∈ whacky
    @show i
end

UndefVarError: UndefVarError: `WhackyRange` not defined

In [14]:
# Mean and std functions tests
test_whack(WhackyRange);

UndefVarError: UndefVarError: `test_whack` not defined

## Problem 5: Rational numbers

Implement the type RationalNumber <: Real, similar to Julia's Rational, but without being a parametric type (i.e. both fields are just Int). Using the following function that simplifies a fraction:

In [10]:
function simplify(n::Integer, d::Integer)
    g = gcd(n, d)
    m = d < 0 ? -1 : 1
    (m * n ÷ g, m * d ÷ g)
end

simplify (generic function with 1 method)

* Define an inner constructor: a function RationalNumber inside the type-definition, which returns new(simplify(n, d)...). new is a special keyword and stands for the type to be created. For example:

In [6]:
struct A
    a::Float64
    function A(a)
        x = cos(a)
        return new(x) # This creates an instance of A with the value x
    end
end


* Inner constructors are used to perform specific tests or transformations necessary before instantiating a type. In this exercise the inner constructor simply ensures that if you try to define the rational number 5/10 you in fact create the equivalent (but simpler) 1/2.

* Then, implement the operations +, -, *, / between RationalNumbers, by extending e.g. Base.:+(r1::RationalNumber, r2::RationalNumber).

* Test your code by ensuring that RationalNumber(5, 10) indeed has fields (1, 2). Then test all operations with the rationals 1/2 and 1/3.


* Find out the minimum extension you need to make to your number type so that all trigonometric functions like cos, tanh, etc. work out of the box with it. (Hint: it is only one more method definition)

In [11]:
# Fill in the Rational struct here
struct RationalNumber <: Real
    numerator::Int
    denominator::Int
end
    
function RationalNumber(n::Integer, d::Integer)
    nn, dd = simplify(n, d)
    return new(nn, dd)
end

RationalNumber

In [12]:
# Fill in the functions here

function +(r1::RationalNumber, r2::RationalNumber)
    num = r1.numerator * r2.denominator + r2.numerator * r1.denominator
    den = r1.denominator * r2.denominator
    return RationalNumber(num, den)
end

function -(r1::RationalNumber, r2::RationalNumber)
    num = r1.numerator * r2.denominator - r2.numerator * r1.denominator
    den = r1.denominator * r2.denominator
    return RationalNumber(num, den)
end

function *(r1::RationalNumber, r2::RationalNumber)
    num = r1.numerator * r2.numerator
    den = r1.denominator * r2.denominator
    return RationalNumber(num, den)
end

function /(r1::RationalNumber, r2::RationalNumber)
    num = r1.numerator * r2.denominator
    den = r1.denominator * r2.numerator
    return RationalNumber(num, den)
end

/ (generic function with 1 method)

In [13]:
RationalNumber(5,10)

RationalNumber(5, 10)